In [1]:
using GridapTopOpt, Gridap

In [2]:
# FE parameters
    order = 1                                                        # Finite element order
    xmax,ymax = (120,40)                                            # Domain size
    dom = (0,xmax,0,ymax)                                            # Bounding domain
    nx,ny = (210,70)
    el_size = (nx,ny)                                              # Mesh partition size
    prop_Γ_N = 0.2                                                   # Γ_N size parameter
    prop_Γ_D = 0.5                                                  # Γ_D size parameter
   f_Γ_N(x) = (x[1] ≈ xmax) &&                                       # Γ_N indicator function
    (0.5*ymax - 5*ymax/ny <= x[2] <= 0.5*ymax + 5*ymax/ny)
f_Γ_D(x) = (x[1] ≈ 0.0)

f_Γ_D (generic function with 1 method)

In [3]:
# FD parameters
    γ = 0.1                                                          # HJ eqn time step coeff
    γ_reinit = 0.5                                                   # Reinit. eqn time step coeff
    max_steps = 20#floor(Int,order*minimum(el_size)/10)                 # Max steps for advection
    tol = 1e-6#1/(15order^2)/minimum(el_size)                              # Reinitialisation tolerance

1.0e-6

In [4]:
 # Problem parameters
    C = isotropic_elast_tensor(2,1.0,0.3)                            #isotropic_elast_tensor(dimension,E,ν)
    g = VectorValue(0,-1)                                            # Heat flow in
    vf = 0.3                                                         # Volume fraction constraint
    lsf_func = initial_lsf((12/xmax),0.2)                                    # Initial level set function
    iter_mod = 10                                                    # VTK Output modulo
    path = "./Result/tolerance&Max_stepModification/results_200times100/N_0.5/40times120/comp_serial/E_1/"
    mkpath(path)                                                     # Create path

"./Result/tolerance&Max_stepModification/results_200times100/N_0.5/40times120/comp_serial/E_1"

In [5]:
# Model
    model = CartesianDiscreteModel(dom,el_size);
    update_labels!(1,model,f_Γ_D,"Gamma_D")
    update_labels!(2,model,f_Γ_N,"Gamma_N")

In [6]:
# Triangulation and measures
    Ω = Triangulation(model)
    Γ_N = BoundaryTriangulation(model,tags="Gamma_N")
    dΩ = Measure(Ω,2*order)
    dΓ_N = Measure(Γ_N,2*order)

GenericMeasure()

In [7]:
# Spaces
    reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
    reffe_scalar = ReferenceFE(lagrangian,Float64,order)
    V = TestFESpace(model,reffe;dirichlet_tags=["Gamma_D"])
    U = TrialFESpace(V,VectorValue(0.0,0.0))
    V_φ = TestFESpace(model,reffe_scalar)
    V_reg = TestFESpace(model,reffe_scalar;dirichlet_tags=["Gamma_N"])
    U_reg = TrialFESpace(V_reg,0)

TrialFESpace()

In [ ]:
   # Level set and interpolator
    φh = interpolate(lsf_func,V_φ)
    interp = SmoothErsatzMaterialInterpolation(η = 2*maximum(get_el_Δ(model)))    # η = 2 ×  maximum side length of an element.
    I,H,DH,ρ = interp.I,interp.H,interp.DH,interp.ρ
    # Weak formulation
    a(u,v,φ) = ∫((I ∘ φ)*(C ⊙ ε(u) ⊙ ε(v)))dΩ
    l(v,φ) = ∫(v·g)dΓ_N
    state_map = AffineFEStateMap(a,l,U,V,V_φ)#,U_reg,φh,dΩ,dΓ_N)
    # Objective and constraints
    J(u,φ) = ∫((I ∘ φ)*(C ⊙ ε(u) ⊙ ε(u)))dΩ
    dJ(q,u,φ) = ∫((C ⊙ ε(u) ⊙ ε(u))*q*(DH ∘ φ)*(norm ∘ ∇(φ)))dΩ
    vol_D = sum(∫(1)dΩ)
    C1(u,φ) = ∫(((ρ ∘ φ) - vf)/vol_D)dΩ
    dC1(q,u,φ) = ∫(-1/vol_D*q*(DH ∘ φ)*(norm ∘ ∇(φ)))dΩ
    pcfs = PDEConstrainedFunctionals(J,[C1],state_map,analytic_dJ=dJ,analytic_dC=[dC1])
    # Velocity extension
    α = 4max_steps*γ*maximum(get_el_Δ(model))
      a_hilb(p,q) = ∫(α^2*∇(p)⋅∇(q) + p*q)dΩ;
    vel_ext = VelocityExtension(a_hilb,U_reg,V_reg)
    evo = FiniteDifferenceEvolver(FirstOrderStencil(2,Float64),model,V_φ;max_steps)
  reinit = FiniteDifferenceReinitialiser(FirstOrderStencil(2,Float64),model,V_φ;tol,γ_reinit)
  ls_evo = LevelSetEvolution(evo,reinit)
    
    # Optimiser
  optimiser = AugmentedLagrangian(pcfs,ls_evo,vel_ext,φh;γ,verbose=true,constraint_names=[:Vol])

AugmentedLagrangian

In [9]:
# Solve
    for (it,uh,φh) in optimiser
        data = ["φ"=>φh,"H(φ)"=>(H ∘ φh),"|∇(φ)|"=>(norm ∘ ∇(φh)),"uh"=>uh]
        iszero(it % iter_mod) && writevtk(Ω,path*"out$it",cellfields=data)
        write_history(path*"/history_$max_steps,$tol.txt",get_history(optimiser))
    end 

Iteration:   0 | L=1.1642e+04, J=1.1291e+04, Vol=3.8683e-01, γ=1.0000e-01, λ1=0.0000e+00, Λ1=4.6931e+03
Iteration:   1 | L=7.9381e+03, J=7.4682e+03, Vol=4.4749e-01, γ=1.0000e-01, λ1=-2.1001e+03, Λ1=4.6931e+03
Iteration:   2 | L=7.2318e+03, J=5.6216e+03, Vol=4.9403e-01, γ=1.0000e-01, λ1=-4.4187e+03, Λ1=4.6931e+03
Iteration:   3 | L=7.7252e+03, J=4.7737e+03, Vol=5.2282e-01, γ=1.0000e-01, λ1=-6.8723e+03, Λ1=4.6931e+03
Iteration:   4 | L=8.7939e+03, J=4.4109e+03, Vol=5.3869e-01, γ=1.0000e-01, λ1=-9.4004e+03, Λ1=4.6931e+03
Iteration:   5 | L=9.7597e+03, J=4.3968e+03, Vol=5.0647e-01, γ=1.0000e-01, λ1=-1.1777e+04, Λ1=5.1624e+03
Iteration:   6 | L=1.0546e+04, J=4.5469e+03, Vol=4.6252e-01, γ=1.0000e-01, λ1=-1.4165e+04, Λ1=5.1624e+03
Iteration:   7 | L=1.1088e+04, J=4.7154e+03, Vol=4.1804e-01, γ=1.0000e-01, λ1=-1.6323e+04, Λ1=5.1624e+03
Iteration:   8 | L=1.1799e+04, J=5.7541e+03, Vol=3.5084e-01, γ=1.0000e-01, λ1=-1.8134e+04, Λ1=5.1624e+03
Iteration:   9 | L=1.2109e+04, J=5.2305e+03, Vol=3.6078e

In [10]:
# Final structure
    it = get_history(optimiser).niter; uh = get_state(pcfs)
    writevtk(Ω,path*"out$it,$max_steps,$tol",cellfields=["φ"=>φh,"H(φ)"=>(H ∘ φh),"|∇(φ)|"=>(norm ∘ ∇(φh)),"uh"=>uh])

┌ Warning: detected extension '.0e-6' does not correspond to dataset type.
│ Appending '.vtu' to filename.
└ @ WriteVTK C:\Users\amiya\.julia\packages\WriteVTK\Be3qm\src\WriteVTK.jl:162


(["./Result/tolerance&Max_stepModification/results_200times100/N_0.5/40times120/comp_serial/E_1/out135,20,1.0e-6.vtu"],)